원천 데이터와 라벨링 데이터가 있는데

Part1은 제목과 본문이 불일치 기사이며 Part2는 본문의 도메인 일관성 부족 기사이다.

CA = 낚시성_자동생성, CD = 낚시성_직접생성, NA = 비낚시성_자동생성이다

In [1]:
import os
import zipfile
import json
import pandas as pd

from sklearn.model_selection import train_test_split

In [2]:
Training_raw_path = 'C:/Users/user/Desktop/과제/data/146.낚시성 기사 탐지 데이터/01.데이터/Training/01.원천데이터/'
Training_labeling_path = 'C:/Users/user/Desktop/과제/data/146.낚시성 기사 탐지 데이터/01.데이터/Training/02.라벨링데이터/'
Validation_raw_path = 'C:/Users/user/Desktop/과제/data/146.낚시성 기사 탐지 데이터/01.데이터/Validation/01.원천데이터/'
Validation_labeling_path = 'C:/Users/user/Desktop/과제/data/146.낚시성 기사 탐지 데이터/01.데이터/Validation/02.라벨링데이터/'

In [3]:
Training_raw_list = []
Training_labeling_list = []
Validation_raw_list = []
Validation_labeling_list = []

In [4]:

# 데이터 다 읽어오는 함수
# 근데 AI 허브 홈페이지에서랑 내가 읽는 파일의 갯수랑 다르다...
def process_read_file(directory_path, data_list):
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.zip'):
            zip_file_path = os.path.join(directory_path, file_name)

            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                # ZIP 파일 내의 모든 파일 목록
                file_list = zip_ref.namelist()

                # 각 JSON 파일에 접근
                for file_name in file_list:
                    # JSON 파일인 경우에만 처리
                    if file_name.endswith('.json'):
                        with zip_ref.open(file_name) as json_file:
                            # JSON 파일 읽기
                            json_data = json.load(json_file)

                            # 필요한 정보에 접근
                            news_id = json_data['sourceDataInfo']['newsID']
                            news_type = json_data['sourceDataInfo']["useType"] # 1 = 진실,  2 = 거짓
                            news_title = json_data['sourceDataInfo']['newsTitle']
                            news_content = json_data['sourceDataInfo']['newsContent']
                            news_category = json_data['sourceDataInfo']['newsCategory']
                            news_sentence_count = json_data['sourceDataInfo']['sentenceCount']
                            
                            #라벨링 데이터는 'clickbaitClass 추가'
                            if 'labeledDataInfo' in json_data and 'clickbaitClass' in json_data['labeledDataInfo']:
                                clickbaitClass = json_data['labeledDataInfo']['clickbaitClass']
                                
                                data_list.append({
                                    'FileName': news_id,
                                    'NewsType' : news_type,
                                    'NewsTitle': news_title,
                                    'NewsCategory' : news_category,
                                    'NewsContent': news_content,
                                    'SentenceCount' : news_sentence_count,
                                    'clickbaitClass' : clickbaitClass,
                                })
                            #raw 데이터
                            else:
                                data_list.append({
                                    'FileName': news_id,
                                    'NewsType' : news_type,
                                    'NewsTitle': news_title,
                                    'NewsCategory' : news_category,
                                    'NewsContent': news_content,
                                    'SentenceCount' : news_sentence_count,
                                })
                            


In [5]:
process_read_file(Training_raw_path, Training_raw_list)
process_read_file(Training_labeling_path, Training_labeling_list)
process_read_file(Validation_raw_path, Validation_raw_list)
process_read_file(Validation_labeling_path, Validation_labeling_list)

In [6]:
# 각 데이터를 df로 만들어 주기
Training_raw_df = pd.DataFrame(Training_raw_list)
Training_labeling_df = pd.DataFrame(Training_labeling_list)
Validation_raw_df = pd.DataFrame(Validation_raw_list)
Validation_labeling_df = pd.DataFrame(Validation_labeling_list)

In [7]:
#   AI 허브에서의 데이터 갯수 raw part1 = 364,333, raw part2 = 369,094, validate part1 = 364,333, validate part1 = 369,094
# 완전 다르쥬..
print(len(Training_raw_df), len(Training_labeling_df), len(Validation_raw_df), len(Validation_labeling_df))

429122 586724 53627 73321


In [8]:
#FileName 기준으로 병합
merged_training  = pd.merge(Training_raw_df, Training_labeling_df, on='FileName', how='inner')
merged_validation = pd.merge(Validation_raw_df, Validation_labeling_df, on='FileName', how='inner')

In [9]:
print(len(merged_training), len(merged_validation))

426020 52004


In [10]:
# 시간이 오래 걸려서 샘플링
sampled_training_data = merged_training.sample(frac=0.01, random_state=42)
sampled_validation_data = merged_validation.sample(frac=0.01, random_state=42)

In [11]:
print(len(sampled_training_data), len(sampled_validation_data))

4260 520


In [12]:
# 칼럼을 정리해야하는데 일단 넘김
merged_training.columns

Index(['FileName', 'NewsType_x', 'NewsTitle_x', 'NewsCategory_x',
       'NewsContent_x', 'SentenceCount_x', 'NewsType_y', 'NewsTitle_y',
       'NewsCategory_y', 'NewsContent_y', 'SentenceCount_y', 'clickbaitClass'],
      dtype='object')

In [13]:
# 형태소 분별
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

# 사용자 정의 불용어 목록을 파일에서 읽어와 리스트로 저장
with open('C:/Users/user/Desktop/과제/data/stop_words.txt', 'r', encoding='utf-8') as file:
    stop_words = [line.strip() for line in file]
    
# 형태소 분석을 위한 Konlpy의 Okt 객체 생성
okt = Okt()

# 형태소 분석 함수 정의
def tokenizer(text):
    return okt.morphs(text)

# 전처리: 각 뉴스 기사의 내용을 형태소로 분할하고 불용어를 제거하여 문자열로 재구성
sampled_training_data['NewsContent_x'] = sampled_training_data['NewsContent_x'].apply(lambda x: ' '.join(tokenizer(x)))
sampled_validation_data['NewsContent_x'] = sampled_validation_data['NewsContent_x'].apply(lambda x: ' '.join(tokenizer(x)))

# TF-IDF 벡터화: TfidfVectorizer를 사용하여 텍스트 데이터를 TF-IDF 행렬로 변환
vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=1000)
X_train = vectorizer.fit_transform(sampled_training_data['NewsContent_x'])
X_val = vectorizer.transform(sampled_validation_data['NewsContent_x'])

y_train = sampled_training_data['clickbaitClass']
y_val = sampled_validation_data['clickbaitClass']

C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '미치다', '바꾸어서', '바꿔', '밖에', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것이', '하다', '하면', '하지', '한하다', '할수록', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  warnings.warn(


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

# 토큰화를 위한 Tokenizer 생성
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(sampled_training_data['NewsContent_x'])

# 훈련 데이터와 검증 데이터의 텍스트를 시퀀스로 변환
X_train_sequences = tokenizer.texts_to_sequences(sampled_training_data['NewsContent_x'])
X_val_sequences = tokenizer.texts_to_sequences(sampled_validation_data['NewsContent_x'])

# 시퀀스를 패딩하여 길이를 일정하게 맞춤
X_train_padded = pad_sequences(X_train_sequences, maxlen=100, padding='post', truncating='post')
X_val_padded = pad_sequences(X_val_sequences, maxlen=100, padding='post', truncating='post')

# 모델 정의
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=16, input_length=100))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(X_train_padded, y_train, epochs=5, validation_data=(X_val_padded, y_val))

# 훈련된 모델 평가
y_pred_prob = model.predict(X_val_padded)
y_pred = (y_pred_prob > 0.5).astype(int)  # 확률이 0.5보다 크면 1, 아니면 0으로 이진화

print(classification_report(y_val, y_pred))


Epoch 1/5
134/134 [==============================] - 9s 32ms/step - loss: 0.6512 - accuracy: 0.6491 - val_loss: 0.6306 - val_accuracy: 0.6846
Epoch 2/5
134/134 [==============================] - 3s 25ms/step - loss: 0.6392 - accuracy: 0.6502 - val_loss: 0.6367 - val_accuracy: 0.6865
Epoch 3/5
134/134 [==============================] - 4s 27ms/step - loss: 0.5991 - accuracy: 0.6836 - val_loss: 0.6808 - val_accuracy: 0.5654
Epoch 4/5
134/134 [==============================] - 3s 24ms/step - loss: 0.5311 - accuracy: 0.7415 - val_loss: 0.6950 - val_accuracy: 0.5885
Epoch 5/5
17/17 [==============================] - 1s 9ms/step
              precision    recall  f1-score   support

           0       0.35      0.37      0.36       164
           1       0.70      0.69      0.70       356

    accuracy                           0.59       520
   macro avg       0.53      0.53      0.53       520
weighted avg       0.59      0.59      0.59       520



샘플링, 최대 피쳐 개수, epoch랑 층이 많이 낮아서 accuracy가 많이 낮다.

하이퍼 파라미터만 조정하면 좀 올라가겠는데 학습을 하루 종일 해야한다. 

여기서 라벨링 데이터의 진실, 거짓 뿐만 아니라 각 패턴 별로 원핫 인코딩 진행하고 모델 학습하면 개량 가능성도 있다.